In [25]:
import procedural
import json
import os

master_path = "/home/kongsgaard/git/procedural-py/"
secret_path = os.path.join(master_path, "secrets")
data_path = os.path.join(master_path, "examples", "data")


In [26]:
user, password, url = procedural.utils.get_secrets(secret_path)
client = procedural.Client(url, user, password)
print(f"CLIENT: {client}")


CLIENT: http://localhost:8001 - Authenticated: Yes


In [27]:
project_name = "Jupyter Project"
project_number = 99
project = client.get_or_create("api/project/", {"name": project_name, "number": project_number})
project_id = project.get("uid")

print(f"PROJECT: {project_id[:8]}")
print(json.dumps(project, indent=4))

PROJECT: 733958e7
{
    "uid": "733958e7-cb8e-4c0b-94da-3d8d8e71b4de",
    "name": "Jupyter Project",
    "number": 99,
    "cluster": "2a6812b0-9b95-4336-8819-26889a677b18",
    "path": "/mnt/cluster/projects/99_jupyter-project",
    "path_type": "name",
    "company": "30eb91f9-d82c-4d1a-9370-9461ac531b92"
}


In [28]:
task_name = "Jupyter Parallel CheckMesh"
parent_task = client.get_or_create(
    f"api/project/{project.get('uid')}/task/",
    {"name": task_name},
    {"config": {
        "task_type": "parent",
        "case_dir": "foam"
    }
    })
parent_id = parent_task.get("uid")

print(f"PARENT TASK: {parent_id[:8]}")
print(json.dumps(parent_task, indent=4))


PARENT TASK: 60f4e356
{
    "uid": "60f4e356-a54f-4bca-8b73-8708b3948f1c",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T07:36:29.053787",
    "config": {
        "case_dir": "foam",
        "task_type": "parent"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-parallel-checkmesh_90ee53ab",
    "parent": null,
    "meta_json": {},
    "project": "733958e7-cb8e-4c0b-94da-3d8d8e71b4de",
    "name": "Jupyter Parallel CheckMesh",
    "dependent_on": null
}


In [29]:
stl_path = os.path.join(data_path, "mesh_vwt", "cfdGeom.stl")
file_upload = client.update(f"api/task/{parent_id}/file/foam/constant/triSurface/cfdGeom.stl/",
                            files=open(stl_path, 'r').read())
print(f"UPLOADED FILE: {file_upload.get('file')}")

UPLOADED FILE: foam/constant/triSurface/cfdGeom.stl


In [30]:
write_mesh_path = os.path.join(data_path, "mesh_vwt", "write_mesh.json")
with open(write_mesh_path, 'r') as f:
    mesh_config = json.load(f)

payload = {"name": "Actions", "config": mesh_config, "parent": parent_id}
action_task = client.create(f"/api/project/{project_id}/task/", payload)
action_id = action_task.get("uid")

write_solution_path = os.path.join(data_path, "mesh_vwt", "write_solution.json")
with open(write_solution_path, 'r') as f:
    solution_config = json.load(f)

payload = {"name": "Actions", "status": "pending", "config": solution_config}
client.update(f"/api/project/{project_id}/task/{action_id}/", payload)

print(f"ACTION TASK: {action_id[:8]}")
print(json.dumps(action_task, indent=4))

configs = client.get(f"/api/task/{action_id}/config/")
print(f"TASK CONFIGS FOR ACTION TASK: {action_id[:8]}")
print(json.dumps(configs, indent=4))

ACTION TASK: feb6a638
{
    "uid": "feb6a638-20aa-4189-9032-7bbc17fcdaf3",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T07:36:29.199650",
    "config": {
        "cmd": "cfd.io.tasks.write_mesh",
        "case_dir": "foam",
        "base_mesh": {
            "type": "simpleBox",
            "cell_size": 2,
            "parameters": {
                "z0": "True",
                "square": "True"
            },
            "bounding_box": {
                "max": [
                    35,
                    35,
                    24
                ],
                "min": [
                    -35,
                    -35,
                    0
                ]
            }
        },
        "task_type": "magpy",
        "snappyhex_mesh": {
            "surfaces": {
                "_269c2d39-4173-45a4-8f97-a21f3917d174": {
                    "level": {
                        "max": "2",
                        "min": "2"
       

In [31]:
cpus = [2, 1, 1]
payload = {
    "name": "Mesh",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "cpus": cpus,
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "checkMesh -writeSets vtk"
        ]},
    "parent": parent_id,
    "dependent_on": action_id
}

mesh_task = client.create(f"/api/project/{project_id}/task/", payload)
mesh_id = mesh_task.get("uid")

print(f"MESH TASK: {mesh_id[:8]}")
print(json.dumps(mesh_task, indent=4))


MESH TASK: ed0088c1
{
    "uid": "ed0088c1-53f2-47fa-b702-21e3957eb8fa",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-08-26T07:36:29.356965",
    "config": {
        "cmd": "pipeline",
        "cpus": [
            2,
            1,
            1
        ],
        "case_dir": "foam",
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "checkMesh -writeSets vtk"
        ],
        "task_type": "cfd"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-parallel-checkmesh_90ee53ab",
    "parent": {
        "uid": "60f4e356-a54f-4bca-8b73-8708b3948f1c",
        "name": "Jupyter Parallel CheckMesh"
    },
    "meta_json": {},
    "project": "733958e7-cb8e-4c0b-94da-3d8d8e71b4de",
    "name": "Mesh",
    "dependent_on": {
        "uid": "feb6a638-20aa-4189-9032-7bbc17fcdaf3",
        "name": "Actions"
    }
}


In [32]:
#client.delete(f"api/project/{project_id}/task/{parent_id}/")
#client.delete(f"api/project/{project_id}/")